In [1]:
import os
import json
import pandas as pd
import numpy as np

In [2]:
data_dir = '/opt/ml/detection/dataset'
anno_source = '/opt/ml/detection/dataset/train.json'

custom_json_dir = '/opt/ml/detection/teambase/custom_annos'
extension = '.json'
file_name = 'custom'

by_image = True

In [3]:
with open(anno_source) as json_file:
    source_anns = json.load(json_file)

print(source_anns.keys())
source_info = source_anns['info']
source_licenses = source_anns['licenses']
source_images = source_anns['images']
source_categories = source_anns['categories']
source_annotations = source_anns['annotations']

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])


In [4]:
df_images_origin = pd.json_normalize(source_images)
df_images_origin.head()

,width,height,file_name,license,flickr_url,coco_url,date_captured,id
0,1024,1024,train/0000.jpg,0,None,None,2020-12-26 14:44:23,0
1,1024,1024,train/0001.jpg,0,None,None,2021-01-10 16:30:39,1
2,1024,1024,train/0002.jpg,0,None,None,2020-12-27 17:55:52,2
3,1024,1024,train/0003.jpg,0,None,None,2021-03-31 12:10:36,3
4,1024,1024,train/0004.jpg,0,None,None,2021-01-01 18:52:38,4


In [5]:
df_anno_origin = pd.json_normalize(source_annotations)
df_anno_origin.head()

,image_id,category_id,area,bbox,iscrowd,id
0,0,0,257301.66,"[197.6, 193.7, 547.8, 469.7]",0,0
1,1,3,10402.56,"[0.0, 407.4, 57.6, 180.6]",0,1
2,1,7,26259.36,"[0.0, 455.6, 144.6, 181.6]",0,2
3,1,4,69096.17,"[722.3, 313.4, 274.3, 251.9]",0,3
4,1,5,24164.58,"[353.2, 671.0, 233.7, 103.4]",0,4


In [6]:
df_anno = df_anno_origin.copy()
df_anno[["X","Y","W","H"]] = df_anno['bbox'].tolist()
df_anno.drop(columns='bbox', inplace=True)
df_anno.head()

,image_id,category_id,area,iscrowd,id,X,Y,W,H
0,0,0,257301.66,0,0,197.6,193.7,547.8,469.7
1,1,3,10402.56,0,1,0.0,407.4,57.6,180.6
2,1,7,26259.36,0,2,0.0,455.6,144.6,181.6
3,1,4,69096.17,0,3,722.3,313.4,274.3,251.9
4,1,5,24164.58,0,4,353.2,671.0,233.7,103.4


In [7]:
# df_anno_img_group = df_anno[['image_id','id']].groupby('image_id').agg(pd.Series.count).reset_index()
# query = df_anno_img_group[df_anno_img_group['id']<=5]
query = df_anno[df_anno['area'] < 10*10]
query.head()
# query의 최종 출력엔 원하는 image_id와, id만 있어도 무방합니다.

,image_id,category_id,area,iscrowd,id,X,Y,W,H
5159,1063,0,0.56,0,5159,647.8,460.2,0.7,0.8
6539,1377,1,3.12,0,6539,719.8,604.7,1.3,2.4
17594,3712,2,66.25,0,17594,883.8,638.5,12.5,5.3


In [8]:
select_imgs, select_annos = query['image_id'].unique(), query['id'].unique()
custom_images = df_images_origin[df_images_origin['id'].isin(select_imgs)].to_dict('records')
if by_image:
    custom_annos = df_anno_origin[df_anno_origin['image_id'].isin(select_imgs)].to_dict('records')
else:
    custom_annos = df_anno_origin[df_anno_origin['id'].isin(select_annos)].to_dict('records')

output_info = source_info
output_licenses = source_licenses
output_images = source_images
output_categories = source_categories
output_annotations = source_annotations

In [9]:
output_images = custom_images
output_annotations = custom_annos

custom_json = {
                'info' : output_info, 
                'licenses' : output_licenses, 
                'images' : output_images, 
                'categories' : output_categories, 
                'annotations' : output_annotations
            }

print(f'selected [{len(custom_json["images"])}] images with [{len(custom_json["annotations"])}] annotations')

selected [3] images with [41] annotations


In [10]:
out_path = os.path.join(custom_json_dir, file_name+extension)   
with open(out_path, 'w') as outfile:
    json.dump(custom_json, outfile, indent=2)